In [27]:
dados  = spark.read.parquet('Files/dados_API/*.parquet')

dados.createOrReplaceTempView('dados')


StatementMeta(, 10fbdf42-4a25-42ba-8391-4fd2dfe22550, 29, Finished, Available, Finished)

In [28]:
dados = spark.sql("""
    select cotacaoCompra as Cotacao,
    cast(dataHoraCotacao as Date) as Data,
    moeda as Moeda 
    from dados 
    order by Data ASC 
""").drop_duplicates(['Moeda', 'Data'])

StatementMeta(, 10fbdf42-4a25-42ba-8391-4fd2dfe22550, 30, Finished, Available, Finished)

In [29]:
dados.show()

StatementMeta(, 10fbdf42-4a25-42ba-8391-4fd2dfe22550, 31, Finished, Available, Finished)

+-------+----------+-----+
|Cotacao|      Data|Moeda|
+-------+----------+-----+
| 3.8539|2025-01-02|  AUD|
| 4.3033|2025-01-02|  CAD|
| 6.8145|2025-01-02|  CHF|
| 0.8551|2025-01-02|  DKK|
| 6.3781|2025-01-02|  EUR|
| 7.6812|2025-01-02|  GBP|
| 0.0395|2025-01-02|  JPY|
|  6.208|2025-01-02|  USD|
| 0.5456|2025-01-02|  NOK|
| 0.5574|2025-01-02|  SEK|
|  3.819|2025-01-03|  AUD|
| 4.2635|2025-01-03|  CAD|
| 6.7578|2025-01-03|  CHF|
| 0.8492|2025-01-03|  DKK|
|  6.333|2025-01-03|  EUR|
| 7.6343|2025-01-03|  GBP|
|0.03911|2025-01-03|  JPY|
| 6.1557|2025-01-03|  USD|
| 0.5412|2025-01-03|  NOK|
| 0.5534|2025-01-03|  SEK|
+-------+----------+-----+
only showing top 20 rows



In [30]:
%%sql
create table if not exists cotacoes (
    Cotacao double,
    Data date,
    Moeda string
)

USING delta 
PARTITIONED by (Moeda)

StatementMeta(, 10fbdf42-4a25-42ba-8391-4fd2dfe22550, 32, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [31]:

# Estudar os comandos marge, when, matched, then, using (usando)
dados.createOrReplaceTempView('dados_novos')

spark.sql("""
    merge into cotacoes as e 
    USING (
        select 
            Cotacao,
            Data,
            Moeda 
        from dados_novos
    ) as n 

    on e.Moeda = n.Moeda and e.Data = n.Data

    WHEN NOT MATCHED THEN 

    insert (Cotacao, Data, Moeda) values (n.Cotacao, n.Data, n.Moeda)   

    """)



StatementMeta(, 10fbdf42-4a25-42ba-8391-4fd2dfe22550, 33, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [32]:
%%sql

-- select  * from dados_novos;
select * from cotacoes;

StatementMeta(, 10fbdf42-4a25-42ba-8391-4fd2dfe22550, 34, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 3 fields>

In [33]:
%%sql
select count(*) from cotacoes

StatementMeta(, 10fbdf42-4a25-42ba-8391-4fd2dfe22550, 35, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>